### Import necessary packages

In [ ]:
%load_ext autoreload
%autoreload 2
import pyreadstat
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
import scipy
import pandas as pd
import seaborn as sns
from scipy.stats import norm, bernoulli
import matplotlib.pyplot as plt
import os, sys
import matplotlib.patheffects as pe
from utils import ols, make_width_coverage_plot, make_budget_plot, get_data, transform_features
import warnings; warnings.simplefilter('ignore')
from tqdm import tqdm

### Import the ACS PUMS dataset

In [ ]:
features = ['AGEP','SCHL','MAR','DIS','ESP','CIT','MIG','MIL','ANC1P','NATIVITY','DEAR','DEYE','DREM','SEX','RAC1P', 'SOCP', 'COW']
ft = np.array(["q", "q", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c"])
income_features, income, employed = get_data(year=2019, features=features, outcome='PINCP')

### Problem setup

Split data into labeled and unlabeled subsets. Compute ground-truth value of the regression coefficient. Specify range of budgets in fractional form $n_b/n$.

In [ ]:
n_all = len(income)
n_tr = 100

income_features_labeled, income_features_unlabeled, income_labeled, Y = train_test_split(income_features, income, train_size=n_tr)
income_labeled = income_labeled.to_numpy()

X = np.stack([income_features_unlabeled['AGEP'].to_numpy(), income_features_unlabeled['SEX'].to_numpy()], axis=1)
Y = Y.to_numpy()
age = income_features['AGEP'].to_numpy()
sex = income_features['SEX'].to_numpy()
theta_true = ols(np.stack([age, sex], axis=1), income.to_numpy())[0]

In [ ]:
budgets = np.linspace(0.01, 0.03, 10)

### Train XGBoost model

Train XGBoost model on labeled data. Additionally train auxiliary model for predicting the magnitude of prediction error. Compute model uncertainty for unlabeled instances.

In [ ]:
income_features_enc, enc = transform_features(income_features, ft)
income_features_labeled = transform_features(income_features_labeled, ft, enc)[0]
income_features_unlabeled = transform_features(income_features_unlabeled, ft, enc)[0]

dtrain = xgb.DMatrix(income_features_labeled, label=income_labeled)
tree = xgb.train({'eta': 0.3, 'max_depth': 7, 'objective': 'reg:absoluteerror'}, dtrain, 100)
Yhat = tree.predict(xgb.DMatrix(income_features_unlabeled))

dtrain = xgb.DMatrix(income_features_labeled, label=np.abs(income_labeled - tree.predict(xgb.DMatrix(income_features_labeled))))
tree_err = xgb.train({'eta': 0.3, 'max_depth': 7, 'objective': 'reg:absoluteerror'}, dtrain, 100)

In [ ]:
Hessian_inv = np.linalg.inv(1/X.shape[0] * X.T @ X)
h = Hessian_inv[:,0]
predicted_errs_init = np.clip(tree_err.predict(xgb.DMatrix(income_features_unlabeled)), 0, np.inf)
uncertainty = np.abs(h.dot(X.T)) * predicted_errs_init
C_init = np.mean(uncertainty)
C = C_init

### Main experiment

Forms dataframe ```df``` with experiment results. The columns in the dataframe are:

- ```lb``` - interval lower bound

- ```ub``` - interval upper bound

- ```interval width``` - equal to ```ub``` - ```lb```

- ```coverage``` - 0/1 indicator of whether or not interval covered target

- ```estimator``` - one of ```active (w/ fine-tuning)```, ```uniform```, or ```active (no fine-tuning)```

- ```$n_b$``` - budget size 

In [ ]:
n = n_all - n_tr
num_trials = 100
alpha = 0.1
tau = 0.5

# fine-tuning params
batch_size = 1000
steps = 200
eta = 0.01
greedy_steps = 500
steps_err = 500
eta_err = 0.001

In [ ]:
results = []
columns = ["lb", "ub", "interval width", "coverage", "estimator", "$n_b$"]
temp_df = pd.DataFrame(np.zeros((3,len(columns))), columns=columns)

for j in tqdm(range(len(budgets))):
    budget = budgets[j]
    budget_window = int(greedy_steps/budget) # how often we use up remaing budget

    for k in tqdm(range(num_trials)):
        perm = np.random.choice(range(n), n, replace=False)
        Y = Y[perm]
        Yhat = Yhat[perm]
        X = X[perm]
        uncertainty = uncertainty[perm]
        income_features_unlabeled = income_features_unlabeled[perm]
        predicted_errs_init = predicted_errs_init[perm]
        increments_active = []
        increments_nofinetune = []
        finetune_inds = []
        num_collected_active = 0
        num_collected_nofinetune = 0
        tree_new = tree.copy()
        tree_err_new = tree_err.copy()
        Yhat_new = tree_new.predict(xgb.DMatrix(income_features_unlabeled))
        predicted_errs = predicted_errs_init
        
        for i in range(n):
            Yhat_curr = Yhat_new[i]
            uncertainty_normed_curr = np.abs(h.dot(X[i])) * predicted_errs[i] / C * budget
            raw_prob = np.clip(uncertainty_normed_curr, 0, np.maximum(0,(i+1)*budget - num_collected_active))
            if i % budget_window >= budget_window - greedy_steps:
                raw_prob = (i+1)*budget - num_collected_active
            prob = (1-tau)*np.clip(raw_prob, 0, 1) + tau*budget
            xi = bernoulli.rvs(prob)
            if xi == 1:
                finetune_inds.append(i)
                num_collected_active +=1
            increments_active.append(float(Yhat_curr + (Y[i] - Yhat_curr)*xi/prob))
                
            if len(finetune_inds) == batch_size:
                finetune_data = xgb.DMatrix(income_features_unlabeled[finetune_inds], label=Y[finetune_inds])
                tree_new = xgb.train({'eta': eta, 'max_depth': 7, 'objective': 'reg:absoluteerror'}, finetune_data, steps, xgb_model=tree_new)
                errs = np.abs(Y[finetune_inds] - tree_new.predict(xgb.DMatrix(income_features_unlabeled[finetune_inds])))
                finetune_data_err = xgb.DMatrix(income_features_unlabeled[finetune_inds], label=errs)
                tree_err_new = xgb.train({'eta': eta_err, 'max_depth': 7, 'objective': 'reg:absoluteerror'}, finetune_data_err, steps_err, xgb_model=tree_err_new)
                predicted_errs = np.clip(tree_err_new.predict(xgb.DMatrix(income_features_unlabeled)), 0, np.inf)
                C = np.mean(np.abs(h.dot(X.T)) * predicted_errs)
                Yhat_new = tree_new.predict(xgb.DMatrix(income_features_unlabeled))
                finetune_inds = []
 
            raw_prob_nofinetune = np.clip(uncertainty[i] / C_init * budget, 0, np.maximum(0,(i+1)*budget - num_collected_nofinetune))
            if i % budget_window >= budget_window - greedy_steps:
                raw_prob_nofinetune = (i+1)*budget - num_collected_nofinetune
            prob_nofinetune = (1-tau)*np.clip(raw_prob_nofinetune, 0, 1) + tau*budget

            # couple sampling decisions to minimize variance in results
            if prob_nofinetune > prob:
                if xi == 1:
                    xi_nofinetune = 1
                else:
                    xi_nofinetune = bernoulli.rvs((prob_nofinetune - prob)/(1-prob))
            else:
                if xi == 0:
                    xi_nofinetune = 0
                else:
                    xi_nofinetune = bernoulli.rvs(prob_nofinetune/prob)
            
            if xi_nofinetune == 1:
                num_collected_nofinetune += 1
                
            increments_nofinetune.append(Yhat[i] + (Y[i] - Yhat[i])*xi_nofinetune/prob_nofinetune)

        active_labels = np.array(increments_active)
        pointest_active = ols(X, active_labels)
        grads = np.zeros(X.shape)
        for i in range(n):
            grads[i,:] = (np.dot(X[i,:], pointest_active) - active_labels[i]) * X[i,:]
        V = np.cov(grads.T)
        Sigma_active = Hessian_inv @ V @ Hessian_inv
        pointest_active_std = np.sqrt(Sigma_active[0,0])/np.sqrt(n)
        width_active = norm.ppf(1-alpha/2)*pointest_active_std 
        coverage_active = (theta_true >= pointest_active[0] - width_active)*(theta_true <= pointest_active[0] + width_active)   
        temp_df.loc[0] = pointest_active[0] - width_active, pointest_active[0] + width_active, 2*width_active, coverage_active, "active (w/ fine-tuning)", int(budget*n)
        
        nofinetune_labels = np.array(increments_nofinetune)
        pointest_nofinetune = ols(X, nofinetune_labels)
        grads = np.zeros(X.shape)
        for i in range(n):
            grads[i,:] = (np.dot(X[i,:], pointest_nofinetune) - nofinetune_labels[i]) * X[i,:]
        V = np.cov(grads.T)
        Sigma_nofinetune = Hessian_inv @ V @ Hessian_inv
        pointest_nofinetune_std = np.sqrt(Sigma_nofinetune[0,0])/np.sqrt(n)
        width_nofinetune = norm.ppf(1-alpha/2)*pointest_nofinetune_std 
        coverage_nofinetune = (theta_true >= pointest_nofinetune[0] - width_nofinetune)*(theta_true <= pointest_nofinetune[0] + width_nofinetune)   
        temp_df.loc[1] = pointest_nofinetune[0] - width_nofinetune, pointest_nofinetune[0] + width_nofinetune, 2*width_nofinetune, coverage_nofinetune, "active (no fine-tuning)", int(budget*n)
        
        xi_unif = bernoulli.rvs([budget]*n)
        unif_labels = np.array(Yhat + (Y - Yhat)*xi_unif/budget)
        pointest_unif = ols(X, unif_labels)
        grads = np.zeros(X.shape)
        for i in range(n):
            grads[i,:] = (np.dot(X[i,:], pointest_unif) - unif_labels[i]) * X[i,:]
        V = np.cov(grads.T)
        Sigma_unif = Hessian_inv @ V @ Hessian_inv
        pointest_unif_std = np.sqrt(Sigma_unif[0,0])/np.sqrt(n)
        width_unif = norm.ppf(1-alpha/2)*pointest_unif_std 
        coverage_unif = (theta_true >= pointest_unif[0] - width_unif)*(theta_true <= pointest_unif[0] + width_unif)
        temp_df.loc[2] = pointest_unif[0] - width_unif, pointest_unif[0] + width_unif, 2*width_unif, coverage_unif, "uniform", int(budget*n)
        
        results += [temp_df.copy()]
df = pd.concat(results,ignore_index=True)

### Plot coverage and interval width

In [ ]:
make_width_coverage_plot(df, "regression coefficient", "widths_and_coverage_census_seq.pdf", theta_true, num_trials=num_trials, n_example_ind=3, finetuning=True, less_precision=True)

### Plot budget saved

In [ ]:
make_budget_plot(df, "Census data analysis", "budget_census_seq.pdf", finetuning=True)